**Entrega 2, Inteligencia Artificial**, UNAL sede Bogota. \\
Integrantes: 
Marco Fidel Caro Durán, 
Alexis Orlando Sánchez Virgüez, 
Nils Peer Beck

# **Ruido en el campus**

---

Dentro de las distintas formas de contaminación que se presentan en el ambiente, está la contaminación acústica. Este tipo de contaminación es generada por el ruido, el cual es todo sonido no deseado con una intensidad alta, que puede ser nocivo para la salud (puede generar perdida del equilibrio y aumento de la presión arterial).

Para estudiar el tema del ruido han sido varias las investigaciones y proyectos, es tal el caso de un grupo de investigadores de la Universidad Politécnica de Madrid (UPM), quienes desarrollaron un modelo  de análisis de texto para  detectar niveles de contaminación acústica, mediante las quejas de los usuarios en las redes sociales. También está el proyecto SONYC, que es una iniciativa de científicos de la Universidad de Nueva York que surgió con el propósito de analizar y mitigar los problemas de ruido de esa ciudad. Como primera fase del proyecto se realiza un monitoreo de la ciudad a través de una red compuesta por sensores y personas, y así se obtiene un reporte de ruido a gran escala; Las fases posteriores analizan los datos y buscan la manera de atenuar el ruido. La tesis doctoral de Natalia Genaro García de la Universidad de Granada (España), desarrolla un modelo basado en expertos en la materia y recogiendo datos de diferentes tipos de calle con el objeto de predecir el ruido ambiental urbano utilizando Redes Neuronales Artificiales (RNA).

Este proyecto no busca en principio dar solución al problema del ruido, lo que se pretende es realizar un mapa del campus de la Universidad Nacional, con los sitios y horarios en los que se presenta con mayor frecuencia. Este mapa podrá ser utilizado posteriormente para quienes desean analizar su impacto y realizar algún plan de acción para poder reducir el daño causado.  


# **Materiales y métodos**

Para el desarrollo del mapa se realizó un censado mediante el uso del teléfono celular, instalando las aplicaciones GPS Logger y Grabador de voz. Se realizaron registros en distintos puntos del campus en intervalos de tiempo de alrededor de una hora; la primera app proporciona datos de ubicación y la segunda realiza una grabación del ruido presente en el lugar. Los datos de ubicación se almacenan en un archivo con formato csv; mientras que los registros de audio se almacenan en formato wav. En  python se toman los archivos de registro y se extrae los datos de interés: del archivo csv se toman los datos del tiempo, latitud y longitud; mientras de que del archivo de audio se realiza un promedio del ruido registrado y se caracteriza por una medida en decibeles (db). Se concatenan los datos extraídos de ambos archivos y se procede a identificarlos en un mapa del campus.

# **Descripción de datos y su almacenamiento**

Debido a que los registros tomados provienen de celulares diferentes para determinar el ruido se hizo un análisis  de la siguiente forma:
Tomamos audios  que  se consideran como ruido y no ruido. A partir de estos datos gereramos datos sintéticos con técnicas de _data augmentation_ para audios. Para generar estos datos sintéticos utilizamos inyección de ruido blanco, agregando valores aleatorios a los datos utilizando la librería _numpy_  y cambios de tiempo corriendo el audio a izquierda o a derecha.
Después se hizo  un análisis de estos audios sintéticos. A partir de su espectro de frecuencia y amplitud tomamos valores mínimos y máximos relativos para cada espectro. Estos valores se guardan en una lista  utilizando la librería _pandas_ y se etiquetan con un 1 si viene de un audio sintético de ruido o un 0 si no. Por último, utilizamos una regresión logística y la entrenamos con la lista generada para identificar ruido.  







# Montar un Drive
Vamos a conectar la carpeta de Google Drive que estamos usando como clase para almacenar las grabaciones

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Instalar software necesario para las librerías

In [0]:
!sudo apt-get install python-pyaudio python3-pyaudio

# Importar liberías

In [0]:
import pyaudio
import os
import csv
import folium
import random
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal
from scipy import fftpack

# Representar los datos
Para poder trabajar de forma eficaz con los datos que grabamos, creamos una clase que represente una grabación y que solamente contiene los datos que nos interesen. Es decir, la ubicación en que fue grabado un data, el tiempo en el que fue grabado y el nivel de ruido que se grabó. \\
Más adelante vamos a poder analizar objetos de esta clase y vizualizarlos en un mapa.

In [0]:
class Audio_sample:
  def __init__(self, latitude, longitude, time, is_loud):
        self.latitude = float(latitude)
        self.longitude = float(longitude)
        self.time = time
        self.is_loud = is_loud

# Extraer coordenadas y tiempo de grabación de los archivos csv

In [0]:
def extract_coords_and_time(csv_file):
  with open(csv_file) as f:
    reader = csv.reader(f)
    for row in reader:
      if row[0] != "time":
        # return latitude, longitude and time values    
        return row[1], row[2], row[0]

# Crear archivos _wav_ de ruido y no ruido para entrenar modelo


In [0]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

class AudioAugmentation:
    def read_audio_file(self, file_path):
        input_length = 6628421
        data = librosa.core.load(file_path)[0]
        print(len(librosa.core.load(file_path)[0]))
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

    def write_audio_file(self, file, data, sample_rate=6628421):
        librosa.output.write_wav(file, data, sample_rate)

    def plot_time_series(self, data):
        fig = plt.figure(figsize=(14, 8))
        plt.title('Raw wave ')
        plt.ylabel('Amplitude')
        plt.plot(np.linspace(0, 1, len(data)), data)
        plt.show()

    def add_noise(self, data):
        noise = np.random.randn(len(data))
        data_noise = data + 0.009 * noise
        return data_noise

    def shift(self, data,i):
        return np.roll(data, 100000 + i*100)

    def stretch(self, data, rate=1):
        input_length = 6628421
        data = librosa.effects.time_stretch(data, rate)
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

# Create a new instance from AudioAugmentation class
for i in range(2):
  aa = AudioAugmentation()

  # Read and show cat sound
  data = aa.read_audio_file("/content/drive/My Drive/data_taller_II_IA/Entrenador/FuelConsumptionCo2.csv")
  aa.plot_time_series(data)

  # Adding noise to sound
  data_noise = aa.add_noise(data)
  aa.plot_time_series(data_noise)

  # Shifting the sound
  data_roll = aa.shift(data,i)
  aa.plot_time_series(data_roll)

  # Stretching the sound
  #data_stretch = aa.stretch(data, 0.8)
  #aa.plot_time_series(data_stretch)

  # Write generated cat sounds
  aa.write_audio_file('/content/drive/My Drive/data_taller_II_IA/Entrenador/test1{}.wav'.format(i), data_noise)
  aa.write_audio_file('/content/drive/My Drive/data_taller_II_IA/Entrenador/test2{}.wav'.format(i), data_roll)
 # aa.write_audio_file('/content/shared-drives/My Drive/audiosdeprueba/test3{}.wav'.format(i), data_stretch)

# Medidor del modelo

In [0]:
 def md(wav_file):
  df=pd.read_csv("/content/drive/My Drive/Entrenador/TEST.csv")  # base da datos para el entrenamiento
  df.rename(columns={df.columns[0]:'x_0',df.columns[1]:'x_1',df.columns[2]:'x_2',df.columns[3]:'x_3',df.columns[4]:'x_4',df.columns[5]:'x_5',df.columns[6]:'x_6',df.columns[7]:'x_7',df.columns[8]:'x_8',df.columns[9]:'x_9',df.columns[10]:'x_10',df.columns[11]:'x_11',df.columns[12]:'x_12',df.columns[13]:'x_13',df.columns[14]:'x_14',df.columns[15]:'x_15',df.columns[16]:'x_16',df.columns[17]:'x_17',df.columns[18]:'x_18',df.columns[19]:'x_19',df.columns[20]:'e'},inplace=True)
  X=df[['x_0','x_1','x_2','x_3','x_4','x_5','x_6','x_7','x_8','x_9','x_10','x_11','x_12','x_13','x_14','x_15','x_16','x_17','x_18','x_19']] #almacenamos datos de las variables de entrada 
  Y=df['e'] # almacenamos datos de la variable de salida
  from sklearn.model_selection import train_test_split  #librerías para separar los datos de prueba y entrenamiento
  X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=1) #tomamos un 100% de los datos para obtener el modelo
  Z=Y_train
  from sklearn.linear_model import LogisticRegression #librería para usar el modelo de regresión logística 
  d=LogisticRegression(solver = 'lbfgs') #definimos el algoritmo y definimos un 'solver' para evitar alertas
  d.fit(X_train,Y_train) #entrenamos el modelo con los datos
  L=d.score(X_train,Z)*100 #establece la medida del modelo
  return L

# Determinar si un cierto archivo wav es ruidoso o no

In [0]:
def determine_if_loud(wav_file):
  #return bool(random.getrandbits(1))
  
  rate, audio = wavfile.read(wav_file)
  M=1024
  freqs, times, Sx = signal.spectrogram(audio, fs=rate, window='hanning',
                                      nperseg=1024, noverlap=M - 100,
                                      detrend=False, scaling='spectrum')
  T=int(rate/5)
  W=int(len(freqs)/5)
  A=np.array(audio)
  B=np.array(freqs)
  f=0
  d=T
  m=0
  n=W
  lista=[]
  for i in range(5):
    P=A[f:d]
    H=B[m:n]
    h=max(P)
    j=min(P)
    k=max(H)
    l=min(H)
    lista.append(h)
    lista.append(j)
    lista.append(k)
    lista.append(l) 
    f=d
    d=d+T
    m=n
    n=n+W    
  df=pd.read_csv("/content/drive/My Drive/Entrenador/TEST.csv")  # base da datos para el entrenamiento
  df.rename(columns={df.columns[0]:'x_0',df.columns[1]:'x_1',df.columns[2]:'x_2',df.columns[3]:'x_3',df.columns[4]:'x_4',df.columns[5]:'x_5',df.columns[6]:'x_6',df.columns[7]:'x_7',df.columns[8]:'x_8',df.columns[9]:'x_9',df.columns[10]:'x_10',df.columns[11]:'x_11',df.columns[12]:'x_12',df.columns[13]:'x_13',df.columns[14]:'x_14',df.columns[15]:'x_15',df.columns[16]:'x_16',df.columns[17]:'x_17',df.columns[18]:'x_18',df.columns[19]:'x_19',df.columns[20]:'e'},inplace=True)
  X=df[['x_0','x_1','x_2','x_3','x_4','x_5','x_6','x_7','x_8','x_9','x_10','x_11','x_12','x_13','x_14','x_15','x_16','x_17','x_18','x_19']] #almacenamos datos de las variables de entrada 
  Y=df['e'] # almacenamos datos de la variable de salida
  from sklearn.model_selection import train_test_split  #librerías para separar los datos de prueba y entrenamiento
  X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=1) #tomamos un 100% de los datos para obtener el modelo
  from sklearn.linear_model import LogisticRegression #librería para usar el modelo de regresión logística 
  d=LogisticRegression(solver = 'lbfgs') #definimos el algoritmo y definimos un 'solver' para evitar alertas
  d.fit(X_train,Y_train) #entrenamos el modelo con los datos
  yhat = d.predict([np.array(lista)])
  if yhat==[1]:
     return True
  else:
     return False

  

# Visualizar los datos con librería folium

In [0]:
def visualize_data(samples):
  # crear un mapa con la ubicación de la primera grabación como centro
  m = folium.Map(location=[samples[0].latitude, samples[0].longitude], zoom_start=16)

  # poner etiquetas en el mapa
  for sample in samples:
    if sample.is_loud:
      sample_color = 'red'
    else:
      sample_color = 'green'
    folium.Marker(location=[sample.latitude, sample.longitude], 
                  icon=folium.Icon(color=sample_color), popup=sample.time).add_to(m)
  return m  

# Ejecutar funciones arriba para procesar los datos y crear la visualización

In [0]:
# data to be stored in the list below
path="/content/drive/My Drive/audios2/"
list_of_samples = []
Lista2=[]

for filename in os.listdir(path):
  if filename.endswith(".wav"): 
    noise_level = determine_if_loud(path + filename)
    Lista2.append(md(path + filename))
    # find corresponding csv file
    for other_file in os.listdir(path):
      if other_file.startswith(filename[:-4]) and other_file.endswith(".csv"):
        # the following command only works with PURE CSV FILES
        csv_params = extract_coords_and_time(path + other_file)
        break
    list_of_samples.append(Audio_sample(csv_params[0], csv_params[1], csv_params[2], noise_level))

y =(sum(np.array(Lista2)))/(len(np.array(Lista2)))
print("La Medida del modelo es de {0:.2f}%".format(y))

visualize_data(list_of_samples)

La Medida del modelo es de 75.26%


# **Resumen del trabajo y limitaciones**
## Logros
En este trabajo hemos creado un sistema que por patrones en sus nombres encuentra a archivos _csv_ y archivos _wav_ que pertenecen juntos. El sistema además los analiza con el fin de crear un mapa de lugares ruidosos y no ruidosos en el campus de la Universidad Nacional de Colombia en Bogotá. Esto implica la creación de una clase de Python que representa a los datos tomados de la forma más simple, es decir, más abstracta posible. Además significa usar una librería para visualizar los datos en un mapa. En este caso usamos _folium_ con este fin. Construimos también un modelo para poder clasificar los archivos _wav_ en ruidoso o no, es decir, en categorías booleanas.

## Limitaciones
Con vista a la propuesta inicial del proyecto, logramos implementarla casi en totalidad. Presentamos procesos funcionales de la colección, del análisis y de la visualisación de datos correspondientes al ruido en determindados lugares geográficos. \\
Sin embargo, el conjunto de datos que usamos para el entrenamiento del modelo es muy pequeño, así que el modelo es bastante limitado en sus capacidades. También nos limitamos a analizar archivos _wav_ largos, es decir de varios minutos hasta horas, como representaciones de un solo momento. Simplificamos estos datos para poder trabajar de forma más sencilla con ellos, pero eso implica también menor calidad de análisis.

## Conclusion
En fin, concluimos haber implementado de forma funcional la propuesta de la clase de Inteligencia Artificial. No obstante, en los detalles con respeto al modelo de aprendizaje de máquina, simplificamos mucho para nuestra causa y recomendamos más precisión si fuera un proyecto de impacto importante.